In [1]:
from neo4j import GraphDatabase

URI = "bolt://localhost:7687"
AUTH = ('neo4j', 'neo4j1234')

In [71]:
def findATrip(srcCity, dstCity):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        query = 'match p=(s:Airport)-->*(d: Airport) where s.city = $src and d.city = $dst return relationships(p), nodes(p) limit 1'
        records, _, _ = driver.execute_query(query, src=srcCity, dst=dstCity)
        driver.close()
        
    #Extract
    trips = []
    for record in records:
        routes = record.get('relationships(p)')
        nodes = record.get('nodes(p)')
        
        path = []
        for i in range(len(routes)):
            src, dst, route = nodes[i], nodes[i + 1], routes[i]
            airline = route.get("airline_name")
            dept = src.get("IATA") + ", " + src.get("city")
            dest = dst.get("IATA") + ", " + dst.get("city")
            path.append(f"{airline} airline: {dept}  to  {dest}")
        
        trips.append(path)
    
    for t in trips:
        print('Trip: ', t)
    return trips


In [72]:
findATrip("Seattle", "Las Vegas")

Trip:  ['WN airline: SEA, Seattle  to  LAS, Las Vegas']


[['WN airline: SEA, Seattle  to  LAS, Las Vegas']]

In [78]:
def findATripWithDHops(srcCity, dstCity, dHops):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        query = "match p=(s:Airport)-->{1," + str(dHops) + "}(d: Airport) where s.city = $src and d.city = $dst return relationships(p), nodes(p) limit 1"
        records, _, _ = driver.execute_query(query, src=srcCity, dst=dstCity)
        driver.close()
        
    #Extract
    trips = []
    for record in records:
        routes = record.get('relationships(p)')
        nodes = record.get('nodes(p)')
        
        path = []
        for i in range(len(routes)):
            src, dst, route = nodes[i], nodes[i + 1], routes[i]
            airline = route.get("airline_name")
            dept = src.get("IATA") + ", " + src.get("city")
            dest = dst.get("IATA") + ", " + dst.get("city")
            path.append(f"{airline} airline: {dept}  to  {dest}")
        
        trips.append(path)
    
    for t in trips:
        print('Trip: ', t)
    return trips


In [79]:
findATripWithDHops("Seattle", "Las Vegas", 10)

Trip:  ['WN airline: SEA, Seattle  to  LAS, Las Vegas']


[['WN airline: SEA, Seattle  to  LAS, Las Vegas']]

In [98]:
def findCitiesWithDHops(srcCity, dHops):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        query = "match p=(s:Airport)-->{1," + str(dHops) + "}(dst: Airport) where s.city = $src return distinct dst.city order by dst.city"
        records, _, _ = driver.execute_query(query, src=srcCity)
        cities = list(map(lambda r: r.get('dst.city'), records))
        driver.close()
    
    print(f"{len(cities)} cities found: ")
    print(cities)
    return cities

In [ ]:
findCitiesWithDHops("Seattle", 2)

In [120]:
def findAdjacentAirportNodesOfCity(city):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        query = 'match (s:Airport)-->(d: Airport) where s.city = $city return d'
        records, _, _ = driver.execute_query(query, city = city)
        driver.close()
    return list(map(lambda r: r.get('d'), records))
        

In [121]:
def findAdjacentAirportNodes(airportNode):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        query = 'match (s:Airport)-->(d: Airport) where s.airport_id = $airport_id return d'
        records, _, _ = driver.execute_query(query, airport_id = airportNode.get("airport_id"))
        driver.close()
    return list(map(lambda r: r.get('d'), records))
        

In [184]:
def findAirportNodeOfCity(city, airportNode):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        query = 'match p=(s:Airport)-->(d: Airport) where s.city = $city and d.airport_id = $airport_id return s'
        records, _, _ = driver.execute_query(query, city = city, airport_id = airportNode.get("airport_id"))
        driver.close()
    return records[0].get('s')
        

In [189]:
def findRouteBetweenAirports(src, dst):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        query = 'match p=(s:Airport)-->(d: Airport) where s.airport_id = $src and d.airport_id = $dst return relationships(p)'
        records, _, _ = driver.execute_query(query, src = src.get("airport_id"), dst = dst.get("airport_id"))
        driver.close()
    return records[0].get('relationships(p)')[0]
        

In [190]:
def bfs(src, dst):
    queue = findAdjacentAirportNodesOfCity(src)
    firstChecks = [d for d in queue if d.get("city") == dst]
    if len(firstChecks) > 0:
        lastNode = {"cur": firstChecks[0], "prev": None}
    else:
        queue = list(map(lambda airport: {"cur": airport, "prev": None}, queue))
        visited = set()
        
        lastNode = None
        while len(queue) > 0:
            top = queue.pop(0)
            curNode = top.get("cur")
            if curNode.get("airport_id") not in visited:
                if curNode.get("city") == dst:
                    lastNode = top
                    break
                
                visited.add(curNode.get("airport_id"))
                for adj in findAdjacentAirportNodes(curNode):
                    if curNode.get("city") == src:
                        continue
                    if adj.get("airport_id") not in visited:
                        queue.append({"cur": adj, "prev": top})
        
    nodes = []
    while lastNode is not None:
        nodes.insert(0, lastNode.get("cur"))
        lastNode = lastNode.get("prev")
    
    if len(nodes) > 0:
        nodes.insert(0, findAirportNodeOfCity(src, nodes[0]))
    # print(nodes)
    
    path = []
    for i in range(len(nodes) - 1):
        src, dst = nodes[i], nodes[i + 1]
        route = findRouteBetweenAirports(src, dst)
        airline = route.get("airline_name")
        dept = src.get("IATA") + ", " + src.get("city")
        dest = dst.get("IATA") + ", " + dst.get("city")
        path.append(f"{airline} airline: {dept}  to  {dest}")
    return path
                    

In [196]:
src = "Seattle"
dst = "Masset"    # Cranbrook

In [197]:
bfs(src, dst)

[<Node element_id='4:4daaf3ee-a640-4fe0-865d-8fe2f4f5e7a0:336' labels=frozenset({'Airport'}) properties={'country': 'United States', 'airport_id': '3577', 'city': 'Seattle', 'IATA': 'SEA', 'name': 'Seattle Tacoma International Airport', 'ICAO': 'KSEA'}>, <Node element_id='4:4daaf3ee-a640-4fe0-865d-8fe2f4f5e7a0:130' labels=frozenset({'Airport'}) properties={'country': 'Canada', 'airport_id': '156', 'city': 'Vancouver', 'IATA': 'YVR', 'name': 'Vancouver International Airport', 'ICAO': 'CYVR'}>, <Node element_id='4:4daaf3ee-a640-4fe0-865d-8fe2f4f5e7a0:856' labels=frozenset({'Airport'}) properties={'country': 'Canada', 'airport_id': '5545', 'city': 'Masset', 'IATA': 'ZMT', 'name': 'Masset Airport', 'ICAO': 'CZMT'}>]


['AC airline: SEA, Seattle  to  YVR, Vancouver',
 '8P airline: YVR, Vancouver  to  ZMT, Masset']